In [4]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_27.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "27_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "27_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "27_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "27_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "27_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "27_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 26_sub_mat_1.csv has been saved.
Submatrix 26_sub_mat_2.csv has been saved.
Submatrix 26_sub_mat_3.csv has been saved.
submatrix 26_cross_matrix_1_2.csv has been saved
submatrix 26_cross_matrix_1_3.csv has been saved
submatrix 26_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,0.719666,-0.810775,0.79611,0.82283,-0.79245,-0.814483,0.754901,-0.592158,0.85957,0.800231,...,-0.432923,-0.819017,-0.83844,0.759648,0.891416,0.816239,-0.86526,-0.80042,0.702657,-0.813656
21085.0,-0.763706,0.742697,0.839369,0.817878,0.810157,0.82619,0.744571,-0.546802,-0.746553,0.831944,...,-0.449996,-0.805103,-0.853267,-0.587729,0.847414,-0.698865,-0.793728,-0.837808,-0.583535,-0.829794
21087.0,0.693564,-0.789261,-0.78497,0.82915,-0.808053,0.798072,0.803182,0.611792,0.788688,0.805881,...,-0.413271,-0.753344,-0.893331,0.630133,0.773403,0.6631,-0.7792,-0.765303,0.543862,-0.882274
21089.0,0.751599,0.800198,0.860063,0.877074,-0.84408,0.858566,0.868763,-0.656678,0.783078,0.884301,...,-0.481212,-0.797148,-0.827129,-0.562631,0.849501,-0.668631,-0.78336,-0.787421,-0.536625,-0.822734
21091.0,0.791839,0.787501,0.881883,0.882774,-0.858802,0.876106,0.856213,-0.63965,-0.830095,0.891264,...,-0.456694,-0.843581,-0.917061,-0.703935,0.826261,-0.802033,-0.848112,-0.846705,-0.640155,-0.914196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.925879,0.951761,0.917553,0.981159,-0.971622,-0.974079,-0.955401,-0.751818,-0.934918,-0.953074,...,-0.508069,-0.942563,-0.934812,-0.653823,-0.956849,-0.77216,-0.951935,-0.944808,-0.534826,-0.827034
38091.0,0.529897,-0.467322,0.498163,0.440346,-0.444696,0.508825,0.453266,0.615311,-0.554201,0.440596,...,0.573607,-0.512509,-0.433302,-0.472877,-0.441158,-0.500202,-0.574552,-0.519472,0.326089,0.409284
38093.0,-0.96774,-0.897866,0.932918,0.929047,-0.987518,-0.980518,0.943959,-0.722511,-0.900511,0.971201,...,-0.535491,-0.901643,-0.999099,-0.619358,-0.922675,-0.751856,-0.907688,-0.948516,-0.569,-0.962805
38095.0,0.676581,0.84834,0.641397,0.714555,0.655285,0.721686,0.716456,-0.823806,0.738736,0.629066,...,-0.830755,0.703076,0.707224,0.636874,0.664269,0.716813,0.694885,0.789522,0.547722,0.644238
